In [ ]:
import pandas as pd

# Step 1: Load data
blinks = pd.read_csv('blinks.csv')
events = pd.read_csv('events.csv')
fixations = pd.read_csv('fixations.csv')

# Step 2: Helper function to get event start and end time
def get_event_times(events_df, internal_event_name):
    start_event = f"{internal_event_name} start"
    end_event = f"{internal_event_name} end"
    try:
        start_time = events_df[events_df['name'] == start_event]['timestamp [ns]'].values[0]
        end_time = events_df[events_df['name'] == end_event]['timestamp [ns]'].values[0]
        return start_time, end_time
    except IndexError:
        return None, None

# Step 3: Event name mapping (for display name to internal file name)
event_name_mapping = {
    'billboard': 'billboard',
    'car following': 'cf',           # Important fix here!
    'pedestrian': 'ped',
    'road closure': 'road',
    'gap acceptance': 'gap',
    'no event': 'no event'
}

# Step 4: Get drive start and recording end times
recording_start = events[events['name'] == 'drive start']['timestamp [ns]'].values[0]
recording_end = events[events['name'] == 'recording.end']['timestamp [ns]'].values[0]

# Step 5: Loop through each event and calculate blinks and fixations
summary = []

for event_display_name in ['entire record'] + list(event_name_mapping.keys()):
    if event_display_name == 'entire record':
        start_time = recording_start
        end_time = recording_end
    else:
        internal_name = event_name_mapping[event_display_name]
        start_time, end_time = get_event_times(events, internal_name)

    if start_time is None or end_time is None:
        continue

    blinks_in_event = blinks[(blinks['start timestamp [ns]'] >= start_time) & (blinks['end timestamp [ns]'] <= end_time)]
    fixations_in_event = fixations[(fixations['start timestamp [ns]'] >= start_time) & (fixations['end timestamp [ns]'] <= end_time)]

    num_blinks = len(blinks_in_event)
    num_fixations = len(fixations_in_event)
    avg_fix_duration = fixations_in_event['duration [ms]'].mean() if num_fixations > 0 else 0

    summary.append({
        'Event': event_display_name,
        'Number of Blinks': num_blinks,
        'Number of Fixations': num_fixations,
        'Average Fixation Duration (ms)': avg_fix_duration
    })

# Step 6: Final summary table
summary_df_corrected = pd.DataFrame(summary)

# (Optional) Save to Excel if needed
summary_df_corrected.to_excel('event_summary.xlsx', index=False)
